In [1]:
from __future__ import division
import time
import torch 
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2 
from util import *
import argparse
import os 
import os.path as osp
from darknet import Darknet
import pickle as pkl
import pandas as pd
import random
import matplotlib.pyplot as plt

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [34]:
plt.rcParams["figure.figsize"] = (15,10)

# Settings

In [3]:
# Replacing the command line arguments with manual arguments here in the notebook.

bs = 1 # "Batch size"
confidence = 0.5 # "Object Confidence to filter predictions"
nms_thresh = 0.4 # "NMS Threshhold"
cfgfile = "cfg/yolov3.cfg"
weightsfile = "yolov3.weights"
reso = 416 # "Input resolution of the network. Increase to increase accuracy. Decrease to increase speed"

#args = arg_parse()
batch_size = int(bs)
confidence = float(confidence)
nms_thesh = float(nms_thresh)
start = 0

CUDA = torch.cuda.is_available()

num_classes = 80
classes = load_classes("data/coco.names.txt")

# YOLOv3 

In [6]:
#Set up the neural network
print("Loading network.....")
model = Darknet(cfgfile)
model.load_weights(weightsfile)
print("Network successfully loaded")

model.net_info["height"] = reso
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()


#Set the model in evaluation mode
model.eval()

Loading network.....
Network successfully loaded


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch

In [57]:
def write(x, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results
    cls = int(x[-1])
    color = random.choice(colors)
    label = "{0}".format(classes[cls])
    cv2.rectangle(img, c1, c2,color, 3)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 3 , 3)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 3, [225,255,255],3);
    return img

# Video Exploration

In [55]:
#Detection phase
videofile ='bball.mp4' #or path to the video file. 

In [41]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(videofile))

In [73]:
def treat_each_image(frame):
 
        #print(frame.shape)
        #plt.imshow(frame)
        
        # Where does this prep image come from?
        img = prep_image(frame, inp_dim)
        
        #print(img.shape)
        
        im_dim = frame.shape[1], frame.shape[0]
        im_dim = torch.FloatTensor(im_dim).repeat(1,2)  
        
        #print(im_dim)
        
        with torch.no_grad():
            output = model(Variable(img), CUDA)
            #print(output)
        output = write_results(output, confidence, num_classes, nms_conf = nms_thesh)
        #print(output.shape)
        
        if type(output) == int:
            #print('here?')
            frames += 1
            #print("FPS of the video is {:5.4f}".format( frames / (time.time() - start)))
            #cv2.imshow("frame", frame)
            key = cv2.waitKey(1)
            #if key & 0xFF == ord('q'):

            #continue
        
        im_dim = im_dim.repeat(output.size(0), 1)
        scaling_factor = torch.min(416/im_dim,1)[0].view(-1,1)
        
        output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim[:,0].view(-1,1))/2
        output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim[:,1].view(-1,1))/2
        
        output[:,1:5] /= scaling_factor

        for i in range(output.shape[0]):
            output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim[i,0])
            output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim[i,1])
    

        classes = load_classes('data/coco.names.txt')
        colors = pkl.load(open("pallete", "rb"))

        list(map(lambda x: write(x, frame), output))
        
        #print(output.shape)
        #print(frame.shape)
        
        #frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #plt.imshow(frame2)
        
        return frame

In [76]:
#clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
clip1 = VideoFileClip("bball.mp4")
output_clip = clip1.fl_image(treat_each_image) #NOTE: this function expects color images!!
%time output_clip.write_videofile("output.mp4", audio=False)

                                                                                                                       
                                                                                                                    

                                                                                                                 


t:   1%|▋                                                                    | 3/308 [20:36<04:59,  1.02it/s, now=None]

t:   1%|▉                                                                    | 4/308 [20:16<05:39,  1.12s/it, now=None]


                                                                                                                       
                                                                                                                    

                                                                                                                 


t:   1%|▋                                        

Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4







t:   1%|▍                                                                    | 2/308 [00:02<06:06,  1.20s/it, now=None]



t:   1%|▋                                                                    | 3/308 [00:04<08:00,  1.57s/it, now=None]



t:   1%|▉                                                                    | 4/308 [00:07<09:05,  1.80s/it, now=None]



t:   2%|█                                                                    | 5/308 [00:09<09:59,  1.98s/it, now=None]



t:   2%|█▎                                                                   | 6/308 [00:12<10:45,  2.14s/it, now=None]



t:   2%|█▌                                                                   | 7/308 [00:14<11:07,  2.22s/it, now=None]



t:   3%|█▊                                                                   | 8/308 [00:17<11:34,  2.32s/it, now=None]



t:   3%|██                                                                   | 9/308 [00:19<12:18,  2.47s/it, now=None]



t:   3%|██▏ 

t:  20%|█████████████▋                                                      | 62/308 [02:18<08:29,  2.07s/it, now=None]



t:  20%|█████████████▉                                                      | 63/308 [02:20<08:29,  2.08s/it, now=None]



t:  21%|██████████████▏                                                     | 64/308 [02:22<08:16,  2.04s/it, now=None]



t:  21%|██████████████▎                                                     | 65/308 [02:24<08:01,  1.98s/it, now=None]



t:  21%|██████████████▌                                                     | 66/308 [02:26<07:44,  1.92s/it, now=None]



t:  22%|██████████████▊                                                     | 67/308 [02:27<07:38,  1.90s/it, now=None]



t:  22%|███████████████                                                     | 68/308 [02:29<07:28,  1.87s/it, now=None]



t:  22%|███████████████▏                                                    | 69/308 [02:31<07:26,  1.87s/it, now=None]



t:  23%|████████

t:  40%|██████████████████████████▌                                        | 122/308 [04:16<05:56,  1.91s/it, now=None]



t:  40%|██████████████████████████▊                                        | 123/308 [04:18<05:54,  1.91s/it, now=None]



t:  40%|██████████████████████████▉                                        | 124/308 [04:20<05:47,  1.89s/it, now=None]



t:  41%|███████████████████████████▏                                       | 125/308 [04:22<05:46,  1.89s/it, now=None]



t:  41%|███████████████████████████▍                                       | 126/308 [04:24<05:40,  1.87s/it, now=None]



t:  41%|███████████████████████████▋                                       | 127/308 [04:25<05:33,  1.84s/it, now=None]



t:  42%|███████████████████████████▊                                       | 128/308 [04:27<05:49,  1.94s/it, now=None]



t:  42%|████████████████████████████                                       | 129/308 [04:30<05:55,  1.99s/it, now=None]



t:  42%|████████

t:  59%|███████████████████████████████████████▌                           | 182/308 [06:14<03:59,  1.90s/it, now=None]



t:  59%|███████████████████████████████████████▊                           | 183/308 [06:16<03:54,  1.88s/it, now=None]



t:  60%|████████████████████████████████████████                           | 184/308 [06:17<03:49,  1.85s/it, now=None]



t:  60%|████████████████████████████████████████▏                          | 185/308 [06:19<03:47,  1.85s/it, now=None]



t:  60%|████████████████████████████████████████▍                          | 186/308 [06:21<03:44,  1.84s/it, now=None]



t:  61%|████████████████████████████████████████▋                          | 187/308 [06:23<03:42,  1.84s/it, now=None]



t:  61%|████████████████████████████████████████▉                          | 188/308 [06:25<03:38,  1.82s/it, now=None]



t:  61%|█████████████████████████████████████████                          | 189/308 [06:27<03:38,  1.84s/it, now=None]



t:  62%|████████

t:  79%|████████████████████████████████████████████████████▋              | 242/308 [08:13<02:28,  2.25s/it, now=None]



t:  79%|████████████████████████████████████████████████████▊              | 243/308 [08:15<02:19,  2.15s/it, now=None]



t:  79%|█████████████████████████████████████████████████████              | 244/308 [08:16<02:12,  2.07s/it, now=None]



t:  80%|█████████████████████████████████████████████████████▎             | 245/308 [08:18<02:08,  2.04s/it, now=None]



t:  80%|█████████████████████████████████████████████████████▌             | 246/308 [08:20<02:03,  1.99s/it, now=None]



t:  80%|█████████████████████████████████████████████████████▋             | 247/308 [08:22<01:58,  1.94s/it, now=None]



t:  81%|█████████████████████████████████████████████████████▉             | 248/308 [08:24<01:55,  1.92s/it, now=None]



t:  81%|██████████████████████████████████████████████████████▏            | 249/308 [08:26<01:58,  2.00s/it, now=None]



t:  81%|████████

t:  98%|█████████████████████████████████████████████████████████████████▋ | 302/308 [10:09<00:11,  1.85s/it, now=None]



t:  98%|█████████████████████████████████████████████████████████████████▉ | 303/308 [10:11<00:09,  1.87s/it, now=None]



t:  99%|██████████████████████████████████████████████████████████████████▏| 304/308 [10:12<00:07,  1.86s/it, now=None]



t:  99%|██████████████████████████████████████████████████████████████████▎| 305/308 [10:14<00:05,  1.85s/it, now=None]



t:  99%|██████████████████████████████████████████████████████████████████▌| 306/308 [10:16<00:03,  1.84s/it, now=None]



t: 100%|██████████████████████████████████████████████████████████████████▊| 307/308 [10:18<00:01,  1.86s/it, now=None]



t: 100%|███████████████████████████████████████████████████████████████████| 308/308 [10:20<00:00,  1.86s/it, now=None]



                                                                                                                       
                   

Moviepy - Done !
Moviepy - video ready output.mp4
Wall time: 10min 24s


In [77]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("output.mp4"))